In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import requests
import urllib
import matplotlib.pyplot as plt

In [ ]:
from TextWorker import *

In [ ]:
separator = r'\', \''
frame_columns_names = ["Тема обращения","Суть обращения",
                        "Чиновник","Должность",
                        "Дата подачи заявления"]
df = pd.read_csv('Mosru_dump_all_pages.csv',
                    encoding = 'utf-8',
                    sep = separator,
                    names = None,
                    skiprows = 1)
df.columns = frame_columns_names

In [ ]:
for column in frame_columns_names:
    df[column] = df[column].apply(clean_string)

In [ ]:
df["Адрес"] = df['Тема обращения'].apply(search_address)

In [ ]:
df["Тема обращения"] = df['Тема обращения'].apply(search_theme)

In [ ]:
df['Обращение нормализованное'] = df['Суть обращения'].apply(preprocessingText)

In [ ]:
df["Дата подачи заявления"] = df['Дата подачи заявления'].apply(prepare_date)

In [ ]:
df.to_csv('clean_frame.csv',sep='|',encoding='utf-8')

Вторая часть после загрузки и обработки данных (подтянуть адреса c помощью mapbox api)

In [ ]:
n_df = pd.read_csv('clean_frame.csv', sep='|', encoding='utf-8')

In [ ]:
unique_addresses = n_df['Адрес'].unique().tolist()

In [ ]:
mapbox_token = "YOUR TOKEN"

In [ ]:
def search_district(address):
    url = "https://api.mapbox.com/geocoding/v5/mapbox.places/{0}.json?access_token={1}".format(urllib.parse.quote("Москва "+str(address)), mapbox_token)
    response = requests.get(url)
    district = ""
    try:
        district = response.json()['features'][0]['place_name']
    except Exception as error:
        print("Возникла ошибка-{0}, адрес - {1}".format(str(error),str(address)) )
    finally:
        return district

In [ ]:
with open('unique_addresses_frame.csv','w',encoding='utf-8') as file:
    file.write('Адрес|Полный ответ\n')
    for address in tqdm(unique_addresses):
        full_response = search_district(address)
        file.write('{0}|{1}\n'.format(address,full_response))

Схлестываем значения адресов с "чистым фреймом"

In [ ]:
addresses_frame = pd.read_csv('unique_addresses_frame.csv',encoding='utf-8',sep='|')

In [ ]:
clean_frame = pd.read_csv('clean_frame.csv', encoding='utf-8',sep='|')

In [ ]:
res_frame = pd.merge(clean_frame, addresses_frame,on="Адрес",how='left')

In [ ]:
res_frame['Округ'] = res_frame['Полный ответ'].apply(search_district)

In [ ]:
res_frame['Проверка округа'] = res_frame[['Округ']].apply(lambda x: 1 if ',' in str(x.values) else 0, axis=1)

In [ ]:
res_adr_frame = res_frame[res_frame['Проверка округа']==0]

Тематическое моделирование

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import nltk

In [ ]:
df = pd.read_csv('clean_frame.csv',encoding='utf-8',sep='|')

In [ ]:
data = df['Обращение нормализованное'].values.tolist()

In [ ]:
data_words = [nltk.word_tokenize(str(line)) for line in data]

In [ ]:
id2word = corpora.Dictionary(data_words)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.LdaMulticore(corpus = corpus, id2word=id2word, num_topics = num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_words, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
num_topics = 3 #количество тем
lda_model = gensim.models.LdaMulticore(corpus = corpus, id2word=id2word, num_topics = num_topics)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

Работа с картой и итоговые графики

In [ ]:
import geopandas as gpd
from MapWorker import *

In [ ]:
df = gpd.read_file("mo.geojson")

In [ ]:
df['NAME'] = df['NAME'].apply( lambda x: x.replace('ё', 'е') )

In [ ]:
df['NAME'] = df['NAME'].apply(lambda x: x.capitalize())

In [ ]:
res_adr_frame['NAME'] = res_adr_frame['Округ'].apply(lambda x: x.capitalize())

In [ ]:
def create_statistic_df(df,column_for_metric,headers):
    fdf = df[column_for_metric].value_counts().to_frame()
    res_df = pd.DataFrame(fdf)
    res_df = res_df.reset_index()
    res_df.columns = headers
    return res_df

In [ ]:
df_districts_counts = create_statistic_df(res_adr_frame,'NAME',['NAME','COUNT'])

In [ ]:
df_districts_counts.plot(x='NAME', y='COUNT', kind='barh',figsize=(15,30))

In [ ]:
districts_statistic = pd.merge(df,df_districts_counts, on='NAME', how='left')

In [ ]:
districts_statistic[['COUNT']] = districts_statistic[['COUNT']].fillna(value=0)

In [ ]:
map_obj = GeoMap(55.75,37.61)

In [ ]:
map_obj.create_coropleth(districts_statistic,
                        districts_statistic,
                        ['NAME','COUNT'],
                        'feature.properties.NAME',
                        'Количество обращений по округам')

In [ ]:
map_obj.add_info_on_map(districts_statistic,
                        ['NAME','COUNT'],
                        ['Название округа','Количество обращений'])

In [ ]:
map_obj.geo_map

In [ ]:
res_adr_frame['Год'] = res_adr_frame['Дата подачи заявления'].apply(
                                                    lambda x: (str(x).split(' ')[0]).split('.')[2])

In [ ]:
years = create_statistic_df(res_adr_frame,'Год',['Год','Количество обращений'])

In [ ]:
years.plot(x='Год', y='Количество обращений', kind='bar')

In [ ]:
applications = create_statistic_df(res_adr_frame,'Тема обращения',['Обращение','Количество'])

In [ ]:
applications[:10].plot(x='Обращение',y='Количество', kind='barh')

In [ ]:
chin = create_statistic_df(res_adr_frame, 'Чиновник',['Чиновник','Количество обращений'])

In [ ]:
chin[:10].plot(x='Чиновник',y='Количество обращений',kind='barh')